In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
import re 
import scipy
from scipy import sparse
import gc 
from IPython.display import display, HTML
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")


pd.options.display.max_colwidth=300

In [2]:
import pickle
from joblib import dump,load

In [3]:
#df_val = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv')
df_val = pd.read_csv('/home/xiaoguzai/数据/Kaggle Jigsaw Rate Severity of Toxic Comments/validation_true.csv')
df_sub = pd.read_csv("/home/xiaoguzai/数据/Kaggle Jigsaw Rate Severity of Toxic Comments/comments_to_score.csv")

## 导入线性模型

## 思路2：切分roberta数据，融合的时候取出0.840+融合，保证数据不重复

In [4]:
n_folds = 7
val_preds_arr1 = np.zeros((df_val.shape[0], n_folds))
val_preds_arr2 = np.zeros((df_val.shape[0], n_folds))
test_preds_arr = np.zeros((df_sub.shape[0], n_folds))

In [5]:
for fld in range(n_folds):
    pipeline = load('/home/xiaoguzai/模型/toxic-linear/withclean'+str(fld)+'.joblib')
    
    # What are the important features for toxicity

    print('\nTotal number of features:', len(pipeline['features'].get_feature_names()) )

    feature_wts = sorted(list(zip(pipeline['features'].get_feature_names(), 
                                  np.round(pipeline['clf'].coef_,2) )), 
                         key = lambda x:x[1], 
                         reverse=True)

    #pprint(feature_wts[:30])
    
    print("\npredict validation data ")
    val_preds_arr1[:,fld] = pipeline.predict(df_val['less_toxic'])
    val_preds_arr2[:,fld] = pipeline.predict(df_val['more_toxic'])
    
    print("\npredict test data ")
    test_preds_arr[:,fld] = pipeline.predict(df_sub['text'])


Total number of features: 75032

predict validation data 

predict test data 

Total number of features: 74635

predict validation data 

predict test data 

Total number of features: 74452

predict validation data 

predict test data 

Total number of features: 75816

predict validation data 

predict test data 

Total number of features: 74327

predict validation data 

predict test data 

Total number of features: 75102

predict validation data 

predict test data 

Total number of features: 74475

predict validation data 

predict test data 


In [6]:
val_preds_arr1c = np.zeros((df_val.shape[0], n_folds))
val_preds_arr2c = np.zeros((df_val.shape[0], n_folds))
test_preds_arrc = np.zeros((df_sub.shape[0], n_folds))

In [7]:
for fld in range(n_folds):
    pipeline = load('/home/xiaoguzai/模型/toxic-linear/withoutclean'+str(fld)+'.joblib')
    
    # What are the important features for toxicity

    #print('\nTotal number of features:', len(pipeline['features'].get_feature_names()) )

    feature_wts = sorted(list(zip(pipeline['features'].get_feature_names(), 
                                  np.round(pipeline['clf'].coef_,2) )), 
                         key = lambda x:x[1], 
                         reverse=True)

    #pprint(feature_wts[:30])
    
    print("\npredict validation data ")
    val_preds_arr1c[:,fld] = pipeline.predict(df_val['less_toxic'])
    val_preds_arr2c[:,fld] = pipeline.predict(df_val['more_toxic'])
    
    print("\npredict test data ")
    test_preds_arrc[:,fld] = pipeline.predict(df_sub['text'])


predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 


In [8]:
val_preds_arr1_ = np.zeros((df_val.shape[0], n_folds))
val_preds_arr2_ = np.zeros((df_val.shape[0], n_folds))
test_preds_arr_ = np.zeros((df_sub.shape[0], n_folds))

for fld in range(n_folds):
    pipeline = load('/home/xiaoguzai/模型/toxic-linear/withruddit'+str(fld)+'.joblib')
    feature_wts = sorted(list(zip(pipeline['features'].get_feature_names(), 
                                  np.round(pipeline['clf'].coef_,2) )), 
                         key = lambda x:x[1], 
                         reverse=True)

    #pprint(feature_wts[:30])
    print("\npredict validation data ")
    val_preds_arr1_[:,fld] = pipeline.predict(df_val['less_toxic'])
    val_preds_arr2_[:,fld] = pipeline.predict(df_val['more_toxic'])

    print("\npredict test data ")
    test_preds_arr_[:,fld] = pipeline.predict(df_sub['text'])


predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 

predict validation data 

predict test data 


In [9]:
print(" Toxic data ")
p1 = val_preds_arr1.mean(axis=1)
p2 = val_preds_arr2.mean(axis=1)

print(f'Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}')

print(" Ruddit data ")
p3 = val_preds_arr1_.mean(axis=1)
p4 = val_preds_arr2_.mean(axis=1)

print(f'Validation Accuracy is { np.round((p3 < p4).mean() * 100,2)}')

print(" Toxic CLEAN data ")
p5 = val_preds_arr1c.mean(axis=1)
p6 = val_preds_arr2c.mean(axis=1)

print(f'Validation Accuracy is { np.round((p5 < p6).mean() * 100,2)}')

 Toxic data 
Validation Accuracy is 82.54
 Ruddit data 
Validation Accuracy is 73.49
 Toxic CLEAN data 
Validation Accuracy is 83.81


In [10]:


print("Find right weight linear")

wts_acc = []
for i in range(30,70,1):
    for j in range(0,20,1):
        w1 = i/100
        w2 = (100 - i - j)/100
        w3 = (1 - w1 - w2 )
        p1_wt = w1*p1 + w2*p3 + w3*p5
        p2_wt = w1*p2 + w2*p4 + w3*p6
        wts_acc.append( (w1,w2,w3, 
                         np.round((p1_wt < p2_wt).mean() * 100,2))
                      )
sorted(wts_acc, key=lambda x:x[3], reverse=True)[:5]



Find right weight linear


[(0.68, 0.13, 0.18999999999999995, 83.29),
 (0.69, 0.13, 0.18000000000000005, 83.29),
 (0.69, 0.12, 0.19000000000000006, 83.29),
 (0.67, 0.14, 0.18999999999999995, 83.27),
 (0.65, 0.16, 0.18999999999999997, 83.25)]

In [11]:
w1,w2,w3,_ = sorted(wts_acc, key=lambda x:x[3], reverse=True)[0]
#print(best_wts)

p1_wt = w1*p1 + w2*p3 + w3*p5
p2_wt = w1*p2 + w2*p4 + w3*p6

In [12]:
## roberta模型调用

In [13]:
import pythonicforbert
from pythonicforbert import get_model_function,FullTokenizer
import torch.nn as nn
from pythonicforbert import get_model_function,FullTokenizer
class ClassificationModel(nn.Module):
    def __init__(self,model,config,n_labels):
        super(ClassificationModel,self).__init__()
        self.model = model
        self.fc = nn.Linear(config.embedding_size,n_labels)
        
    def forward(self,input_ids):
        mask_ids = torch.not_equal(input_ids,1)
        #英文roberta padding=1
        output = self.model(input_ids)
        if mask_ids is not None:
            mask_ids = mask_ids[:,:,None].float()
            output -= 1e-12*(1.0-mask_ids)
        output = output[:,0]
        output = self.fc(output)
        return output

In [14]:
from torch.utils.data import Dataset,DataLoader
from transformers import RobertaTokenizer
vocab_file = '/home/xiaoguzai/模型/roberta-english-base/vocab.json'
 
merges_file = '/home/xiaoguzai/模型/roberta-english-base/merges.txt'
tokenizer = RobertaTokenizer(vocab_file, merges_file)
#tokenizer = RobertaTokenizer.from_pretrained('../input/roberta-base')
class TestDataset(Dataset):
    def __init__(self,text,maxlen):
        self.text = text
        self.maxlen = maxlen
        token_id = []
        for index in tqdm(range(len(self.text))):
            current_text = text[index]
            current_id = tokenizer(current_text)['input_ids']
            #roberta begin:0,end
            #current_id = tokenizer.convert_tokens_to_ids(current_token)
            current_id = self.sequence_padding(current_id)
            token_id.append(current_id)
        self.token_id = token_id
        
    def __len__(self):
        return len(self.token_id)

    def __getitem__(self,index):
        #return [tuple(tensor[index] for tensor in self.tensors)]
        return self.token_id[index]
    
    def sequence_padding(self,inputs,padding=1):
        #英文roberta padding=1
        length = self.maxlen
        if len(inputs) > length:
            inputs = inputs[:length-1]+[inputs[-1]]
        #保留[sep]标志部分
        outputs = []
        pad_width = (0,length-len(inputs))
        x = np.pad(inputs,pad_width,'constant',constant_values=padding)
        return x

In [15]:
import torch
from tqdm import tqdm
import numpy as np
roberta = get_model_function('roberta-base')

In [16]:
lesstoxic_valid_dataset = TestDataset(df_val['less_toxic'],maxlen=500)
lesstoxic_valid_loader = DataLoader(lesstoxic_valid_dataset,batch_size=16)
moretoxic_valid_dataset = TestDataset(df_val['more_toxic'],maxlen=500)
moretoxic_valid_loader = DataLoader(moretoxic_valid_dataset,batch_size=16)
test_dataset = TestDataset(df_sub['text'],maxlen=500)
test_loader = DataLoader(test_dataset,batch_size=16)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

100%|█████████████████████████████████████| 7537/7537 [00:02<00:00, 2751.98it/s]


In [17]:
model = torch.load('/home/xiaoguzai/程序/kaggle有毒评论对比比赛/best_score=0.8418643362463587seed=15.pth')
model = model.to(device)
model.eval()
less_toxic1,more_toxic1 = [],[]
result_toxic1 = []
for batch_token in tqdm(lesstoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        less_toxic1 = less_toxic1+current_point.cpu().numpy().tolist()

for batch_token in tqdm(moretoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        more_toxic1 = more_toxic1+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(test_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        result_toxic1 = result_toxic1+current_point.cpu().numpy().tolist()

100%|█████████████████████████████████████████| 472/472 [00:45<00:00, 10.36it/s]


In [18]:
model = torch.load('/home/xiaoguzai/程序/kaggle有毒评论对比比赛/best_score=0.8416562630045776seed=4.pth')
model = model.to(device)
model.eval()
less_toxic2,more_toxic2 = [],[]
result_toxic2 = []
for batch_token in tqdm(lesstoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        less_toxic2 = less_toxic2+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(moretoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        more_toxic2 = more_toxic2+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(test_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        result_toxic2 = result_toxic2+current_point.cpu().numpy().tolist()

100%|█████████████████████████████████████████| 472/472 [00:45<00:00, 10.29it/s]


In [19]:
model = torch.load('/home/xiaoguzai/程序/kaggle有毒评论对比比赛/best_score=0.8410320432792343seed=5.pth')
model = model.to(device)
model.eval()
less_toxic3,more_toxic3 = [],[]
result_toxic3 = []
for batch_token in tqdm(lesstoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        less_toxic3 = less_toxic3+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(moretoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        more_toxic3 = more_toxic3+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(test_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        result_toxic3 = result_toxic3+current_point.cpu().numpy().tolist()

100%|█████████████████████████████████████████| 472/472 [00:45<00:00, 10.31it/s]


In [20]:
model = torch.load('/home/xiaoguzai/程序/kaggle有毒评论对比比赛/best_score=0.8401997503121099seed=1.pth')
model = model.to(device)
model.eval()
less_toxic4,more_toxic4 = [],[]
result_toxic4 = []
for batch_token in tqdm(lesstoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        less_toxic4 = less_toxic4+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(moretoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        more_toxic4 = more_toxic4+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(test_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        result_toxic4 = result_toxic4+current_point.cpu().numpy().tolist()

100%|█████████████████████████████████████████| 472/472 [00:45<00:00, 10.31it/s]


In [21]:
model = torch.load('/home/xiaoguzai/程序/kaggle有毒评论对比比赛/best_score=0.8395755305867666seed=8.pth')
model = model.to(device)
model.eval()
less_toxic5,more_toxic5 = [],[]
result_toxic5 = []
for batch_token in tqdm(lesstoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        less_toxic5 = less_toxic5+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(moretoxic_valid_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        more_toxic5 = more_toxic5+current_point.cpu().numpy().tolist()
        
for batch_token in tqdm(test_loader):
    batch_token = batch_token.to(device)
    with torch.no_grad():
        current_point = model(batch_token)
        current_point = current_point.squeeze(-1)
        result_toxic5 = result_toxic5+current_point.cpu().numpy().tolist()

100%|█████████████████████████████████████████| 472/472 [00:43<00:00, 10.92it/s]


In [22]:
import time
import multiprocessing
def calculate_data(i,j):
    for k in range(0,31):
        for u in range(0,31):
            v = 100-i-j-k-u
            for z in range(0,30):
                z = z*0.1
                less_toxic_wt = np.array(less_toxic1).dot(i)+np.array(less_toxic2).dot(j)+np.array(less_toxic3).dot(k)+\
                            np.array(less_toxic4).dot(u)+np.array(less_toxic5).dot(v)
                less_toxic_wt = less_toxic_wt+z*p1_wt
                #!!!注意这里的z是乘在哪个部位的 submission时保持一致
                more_toxic_wt = np.array(more_toxic1).dot(i)+np.array(more_toxic2).dot(j)+np.array(more_toxic3).dot(k)+\
                            np.array(more_toxic4).dot(u)+np.array(more_toxic5).dot(v)
                more_toxic_wt = more_toxic_wt+z*p2_wt
                wts_acc.append((i,j,k,u,v,z,
                        np.round((less_toxic_wt < more_toxic_wt).mean() * 100,2))
                          )

In [23]:
wts_acc = multiprocessing.Manager().list()
#思路修正：1.各個相同的模型先分別進行融合，再進行最終的融合
#嘗試方向2.對validation_data進行預訓練
start_time = time.time()
for i in tqdm(range(0,31)):
    for j in range(0,31):
        #pool.apply_async(calculate_data,(i,j,))
        p = multiprocessing.Process(target=calculate_data,args=(i,j,))
        p.start()
        p.join()

sorted(wts_acc, key=lambda x:x[6], reverse=True)[:5]
#新的结果不能使用w1,w2,w3，因为w1,w2,w3要作为最后的结果内容
print('%d second'%(time.time()-start_time))

100%|██████████████████████████████████████| 31/31 [11:59:58<00:00, 1393.51s/it]


43962 second


In [24]:
wts_acc = list(wts_acc)

In [25]:
w4,w5,w6,w7,w8,w,_ = sorted(wts_acc, key=lambda x:x[6], reverse=True)[0]

In [26]:
print('w1 = %f,w2 = %f,w3 = %f,w4 = %f,w5 = %f\n'%(w1,w2,w3,w4,w5))

w1 = 0.680000,w2 = 0.130000,w3 = 0.190000,w4 = 17.000000,w5 = 4.000000

w5 = %f,w6 = 


In [29]:
sorted(wts_acc, key=lambda x:x[6], reverse=True)[:5]

[(17, 4, 30, 11, 38, 2.9000000000000004, 85.58),
 (17, 4, 30, 12, 37, 2.9000000000000004, 85.58),
 (17, 5, 30, 10, 38, 2.9000000000000004, 85.58),
 (18, 4, 30, 10, 38, 2.9000000000000004, 85.58),
 (18, 4, 30, 11, 37, 2.9000000000000004, 85.58)]

In [28]:
df_submission = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv')
df_submission['score'] = w*w1*test_preds_arr.mean(axis=1) + w*w2*test_preds_arr_.mean(axis=1) + w*w3*test_preds_arrc.mean(axis=1) + \
                         np.dot(np.array(result_toxic1),w4) + np.dot(np.array(result_toxic2),w5) + np.dot(np.array(result_toxic3),w6) + \
                         np.dot(np.array(result_toxic4),w7) + np.dot(np.array(result_toxic5),w8) + np.dot(np.array(result_toxic6),w9)
df_submission[['comment_id', 'score']].to_csv("submission.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv'